In [1]:
import pathlib
BASE_DIR=pathlib.Path().resolve().parent 
DATASET_DIR=BASE_DIR/"datasets"
ZIP_DIR=DATASET_DIR/"zips"
ZIP_DIR.mkdir(exist_ok=True, parents=True)

In [2]:
DATASET_ZIP="https://archive.ics.uci.edu/static/public/259/reuters+rcv1+rcv2+multilingual+multiview+text+categorization+test+collection.zip"


In [3]:
!curl  https://archive.ics.uci.edu/static/public/259/reuters+rcv1+rcv2+multilingual+multiview+text+categorization+test+collection.zip  -o $ZIP_DIR/translation-dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 44981    0 44981    0     0  27181      0 --:--:--  0:00:01 --:--:-- 27294
100  211k    0  211k    0     0  75129      0 --:--:--  0:00:02 --:--:-- 75299
100  287k    0  287k    0     0  79828      0 --:--:--  0:00:03 --:--:-- 79966
100  383k    0  383k    0     0  85219      0 --:--:--  0:00:04 --:--:-- 85335
100  383k    0  383k    0     0  64173      0 --:--:--  0:00:06 --:--:-- 70354
100  503k    0  503k    0     0  77362      0 --:--:--  0:00:06 --:--:-- 93926
100  639k    0  639k    0     0  86305      0 --:--:--  0:00:07 --:--:-- 93150
100  747k    0  747k    0     0  86698      0 --:--:--  0:00:08 --:--:-- 91642
100  775k    0  775k    0     0  83310      0 --:--

In [ ]:
!pip install  boto3

In [ ]:
import boto3
import os
import pathlib
import pandas as pd
import pickle

In [ ]:
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.models import Model, Sequential

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
DATASET_CSV_PATH = EXPORT_DIR / 'translation-dataset.csv'


In [ ]:
df = pd.read_csv(DATASET_CSV_PATH)
df.head()

In [ ]:
PART_TWO_GUIDE_PATH = GUIDES_DIR / "2 - Convert Dataset into Vectors.ipynb"
TRAINING_DATA_PATH = EXPORT_DIR / 'spam-training-data.pkl'
EXPORT_DIR = pathlib.Path('/datasets/exports/')


In [ ]:
%run "$PART_TWO_GUIDE_PATH"
#will  convert the dataset into vectors

In [ ]:
Extract prepared training dataset results.

data = {}

with open(TRAINING_DATA_PATH, 'rb') as f:
    data = pickle.load(f)

In [ ]:
X_test = data['X_test']
X_train = data['X_train']
y_test = data['y_test']
y_train = data['y_train']
labels_legend_inverted = data['labels_legend_inverted']
legend = data['legend']
max_sequence = data['max_sequence']
max_words = data['max_words']
tokenizer = data['tokenizer']

In [ ]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(MAX_NUM_WORDS, embed_dim, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
print(model.summary())

In [ ]:
batch_size = 32
epochs = 5
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, verbose=1, epochs=epochs)

In [ ]:
MODEL_EXPORT_PATH = EXPORT_DIR / 'TextTranslation-model.h5'
model.save(str(MODEL_EXPORT_PATH))

In [ ]:
import json
metadata = {
    "labels_legend_inverted": labels_legend_inverted,
    "legend": legend,
    "max_sequence": max_sequence,
    "max_words": max_words,
}

METADATA_EXPORT_PATH = EXPORT_DIR / 'spam-classifer-metadata.json'
METADATA_EXPORT_PATH.write_text(json.dumps(metadata, indent=4))

In [ ]:
tokenizer_as_json = tokenizer.to_json()

TOKENIZER_EXPORT_PATH = EXPORT_DIR / 'spam-classifer-tokenizer.json'
TOKENIZER_EXPORT_PATH.write_text(tokenizer_as_json)